In [1]:
import os
import openpyxl
import pandas as pd
import xlrd
from datetime import datetime
import numpy as np
import pandas as pd
import re, os
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn import preprocessing
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Input, Concatenate, BatchNormalization, Activation
from keras.layers import Lambda, Embedding, GRU, Bidirectional, TimeDistributed, concatenate
from keras.models import Model
from keras import optimizers
from keras import backend as K
from keras.layers import Layer
from keras import initializers
from word2vecReader import Word2Vec
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score
import pickle
# from tensorflow.python.keras.optimizers import adam_v2
# from tensorflow.python.keras.optimizers import rmsprop_v2
import tensorflow as tf
from tensorflow import keras
from  keras import layers
from time import process_time
import re
import jieba
from collections import Counter

os.environ["CUDA_VISIBLE_DEVICES"]="0"

K.set_learning_phase(1)
# K.set_learning_phase(True)   # <==
np.random.seed(0)
MAX_SENT_LENGTH = 30###number of words in a sentence
MAX_NB_WORDS = 20000
POST_DIM = 123
INFO_DIM=30
VALIDATION_SPLIT = 0.2

def crop(dimension, start, end):
    # Crops (or slices) a Tensor on a given dimension from start to end
    # example : to crop tensor x[:, :, 5:10]
    # call slice(2, 5, 10) as you want to crop on the second dimension
    def func(x):
        if dimension == 0:
            return x[start: end]
        if dimension == 1:
            return x[:, start: end]
        if dimension == 2:
            return x[:, :, start: end]
        if dimension == 3:
            return x[:, :, :, start: end]
        if dimension == 4:
            return x[:, :, :, :, start: end]

    return Lambda(func)


def myFunc(x):
    if "empety" in x:
        return False
    else:
        return True


def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\\", "", string)
    string = re.sub(r"\'", "", string)
    string = re.sub(r"\"", "", string)
    string = string.strip().lower()
    word_tokens = word_tokenize(string)
    filtered_words = [word for word in word_tokens if word not in stopwords.words('english')]
    return filtered_words


def find_str(s, char):
    index = 0

    if char in s:
        c = char[0]
        for ch in s:
            if ch == c:
                if s[index:index + len(char)] == char:
                    return index

            index += 1


class AttLayer(Layer):
    def __init__(self, attention_dim):
        self.init = initializers.get('normal')
        self.supports_masking = True
        self.attention_dim = attention_dim
        super(AttLayer, self).__init__()

    def build(self, input_shape):
        assert len(input_shape) == 3
        self.W = K.variable(self.init((input_shape[-1], self.attention_dim)))
        self.b = K.variable(self.init((self.attention_dim,)))
        self.u = K.variable(self.init((self.attention_dim, 1)))
        self.trainable_weightss = [self.W, self.b, self.u]
        super(AttLayer, self).build(input_shape)

    def compute_mask(self, inputs, mask=None):
        return mask

    def call(self, x, mask=None):
        # size of x :[batch_size, sel_len, attention_dim]
        # size of u :[batch_size, attention_dim]
        # uit = tanh(xW+b)
        uit = K.tanh(K.bias_add(K.dot(x, self.W), self.b))
        ait = K.dot(uit, self.u)
        ait = K.squeeze(ait, -1)

        ait = K.exp(ait)

        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            ait *= K.cast(mask, K.floatx())
        ait /= K.cast(K.sum(ait, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        ait = K.expand_dims(ait)
        weighted_input = x * ait
        output = K.sum(weighted_input, axis=1)

        return output

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

# 文件夹路径
folder_path = 'C:/Users/Administrator/Desktop/keyan dateset/微博正文'

# 创建包含三个关键字的字典
data_dict = {
    'bid': [],
    'text': [],
    'comment': [],
    'time': [],
    'poster': [],
    'lable':[]
}

# 遍历文件夹中的所有文件
for file_name in os.listdir(folder_path):
    if file_name.endswith('.xlsx'):
        file_path = os.path.join(folder_path, file_name)

        # 打开 Excel 文件
        workbook = openpyxl.load_workbook(file_path)
        sheet = workbook.active

        # 提取数据
        for row in sheet.iter_rows(min_row=2, values_only=True):
            bid = row[1]
            text = row[5]
            likes = row[8]

            # 将数据添加到字典对应的列表中
            data_dict['bid'].append(bid)
            data_dict['text'].append(text)
            data_dict['poster'].append(likes)

        # 关闭 Excel 文件
        workbook.close()

bid_length = len(data_dict['bid'])
for i,bid in enumerate(data_dict['bid']):
    # 构建文件路径
    for root, directories, files in os.walk('C:/Users/Administrator/Desktop/keyan dateset/comment'):
        for file in files:
            file_name = os.path.splitext(file)[0]
            if(str(bid)==str(file_name)):
                file_path = os.path.join(root, file)
                #print(file_path)
                workbook = xlrd.open_workbook(file_path)
                sheet = workbook.sheet_by_index(0)  # 假设要读取第一个工作表

                # 遍历工作表的行
                comment=[]
                time=[]
                lable=sheet.cell_value(1, 5)
                cell = sheet.cell(1, 5)
                lable = cell.value

                # 如果单元格为空，将其视为0
                if cell.ctype == xlrd.XL_CELL_EMPTY:
                    lable = 0

                for row in range(1, sheet.nrows):
                    comment1 = sheet.cell_value(row,2)
                    time1 = sheet.cell_value(row,3)
                    comment.append(comment1)
                    time.append(time1)
                data_dict['lable'].append(lable)
                data_dict['comment'].append(comment)
                data_dict['time'].append(time)
        # 关闭 Excel 文件
                workbook.release_resources()
# 将字典转换为 DataFrame
#print(data_dict)

# 将字典转换为 DataFrame
df = pd.DataFrame(data_dict)

# 删除comment列为空列表的行
df = df[df['comment'].apply(len) > 0]
df = df[df['time'].apply(lambda x: len(x[0]) != 6)]
# 删除comment和time具有相同值的行
df = df[~((df['comment'] == df['time']) & (df['comment'].apply(len) > 0))]
df['comment'] = df['comment'].astype(str)
df['text'] = df['text'].astype(str)
df['text'] = df['text'].apply(lambda x: re.sub(r'\<.*?\>', '', x))  # 去除方括号及其内容
df['comment'] = df['comment'].apply(lambda x: re.sub(r'\<.*?\>', '', x))  # 去除方括号及其内容
# 重置索引
df = df.reset_index(drop=True)

# 打印 DataFrame
df

E:\anaconda\lib\site-packages\keras\backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


,bid,text,comment,time,poster,lable
0,MFl4csvfW,原来大家所恐惧的16号裁判正是令科尔口吐口香糖的那位金州勇士#湖人对战勇士#LLOVE_维...,"['就是他赌球', '人家可是歪嘴战神', '嘴气歪的那位', '所以呢主场给安排凯尔特人不...","[05月06日 22:39, 05月06日 22:26, 05月06日 22:23, 05月...",167,0.0
1,MF8iLkA7H,#湖人vs勇士#@H和30:《偷一个主场就够了》《金州裁判队》《克莱不可能场场这样》《今天湖...,"['勇蜜又如意了是吧', '去年太阳输独行侠第三场时也是铺天盖地的太阳放水论[送花花]', ...","[05月05日 16:22, 05月05日 17:15, 05月05日 15:58, 05月...",79,0.0
2,MF8Qjy6UC,我发了一条这个比赛勇士输不了，不知道为什么很多人理解成我说的是裁判。裁判尺度肯定是会影响比赛...,"['gswin5', '是的，开场就懒懒散散的，加上裁判这么照顾浓眉，那就没得打了', '裁...","[05月05日 20:38, 05月05日 20:28, 05月05日 18:55, 05月...",69,0.0
3,MF7eQ72mG,花了是勇士三分投的准，湖人自己防守差，打得不好，跟裁判也没关系。我也不认可假球的说法，我没觉...,"['湖人球迷急了', '但是湖人没上强度也是真的，感觉就是在投篮训练，互相投篮就被勇士投花了...","[05月05日 11:06, 05月05日 11:06, 05月05日 11:07, 05月...",429,1.0
4,MF7dZ7U1S,这要是格林和施罗德角色互换一下...球迷能批斗谩骂整个季后赛裁判:由于格林有什么什么历史，所...,"['真的容易拉伤内侧肌肉', '施罗德小动作很多的', '詹密看不见系列', '裤蜜看得见？...","[05月05日 11:08, 05月05日 11:43, 05月05日 19:42, 05月...",378,1.0
...,...,...,...,...,...,...
820,Mp3Pmr9CB,2022年，我们在让人窒息的疫情间隙里为了一场又一场幸福奔走。广东、福建、浙江、江苏、安徽、...,"['', '怎么做到的，太牛了！', '祖国大好河山']","[01月20日 09:57, 01月20日 09:50, 01月19日 22:01]",53,0.0
821,MoV7e2LHs,胡明轩小胡签名签到忘我，上了辽宁队的车，小胡：赶紧溜，你们啥也没看见#胡明轩##广东宏远#...,"['哈哈哈哈哈哈哈你的笔拿回来了吗', '🐯：快跑丢死个人', '对不起哥但是还是想笑哈哈哈...","[01月18日 22:52, 01月18日 22:56, 01月18日 22:59, 01月...",557,0.0
822,MoZqCfoA5,辽粤大战｜郭艾伦14分6篮板6助攻，张镇麟10分4篮板3助攻2断｜广东VS辽宁｜CBA常规赛...,['是的，杜峰太了解了他了，只要郭艾伦持球就知道他下一个动作了，所以真的跟困难，虽然说不能找...,"[01月19日 13:56, 01月19日 11:43, 01月19日 11:05, 01月...",55,0.0
823,MpvTn2kBR,今年只有央视春晚的观众没有戴口罩，其它地方台（包括北京卫视，上海卫视，辽宁卫视，东方卫视，广...,"['地方提前录的，那时候不清楚央视态度，求稳就戴着了', '地方台提前更早录好的吧', '不...","[01月22日 21:27, 01月22日 20:28, 01月23日 08:40, 01月...",268,0.0


In [2]:
time=df['time']

time1_list=[]

In [3]:
for time_list in time:
    if len(time_list[0]) > 6:
        base_time = datetime.strptime(time_list[0], '%m月%d日 %H:%M')
    else:
        base_time = datetime.strptime(time_list[0] + ' 00:00', '%m月%d日 %H:%M')
    time_diff_list = []
    for time_str in time_list:
        if len(time_str) > 8:
            current_time = datetime.strptime(time_str, '%m月%d日 %H:%M')
        else:
            current_time = datetime.strptime(time_str + ' 00:00', '%m月%d日 %H:%M')
        time_diff = (current_time - base_time).total_seconds()
        time_diff_list.append(time_diff)
    time1_list.append(time_diff_list)
print(time1_list)
                
    

[[0.0, -780.0, -960.0, 4860.0, 4860.0, 4860.0, 4860.0, 3660.0, 2700.0, 2400.0, 1800.0, 1200.0, 1080.0, 1020.0, 900.0, 0.0, -60.0, -780.0, -960.0, -960.0, -1200.0], [0.0, 3180.0, -1440.0, 78780.0, 62100.0, 31440.0, 30360.0, 28800.0, 3180.0, 1740.0, 1680.0, 420.0, 0.0, -1440.0, -1620.0, -5700.0, -6180.0, -8880.0, -9060.0, -9180.0], [0.0, -600.0, -6180.0, -6240.0, -17580.0, -17640.0, -18480.0, -19200.0], [0.0, 0.0, 60.0, 78060.0, 75480.0, 58200.0, 48480.0, 42360.0, 33900.0, 31860.0, 31080.0, 23940.0, 23760.0, 22320.0, 22260.0, 22260.0, 22260.0, 22200.0, 18660.0, 11820.0, 11160.0, 10860.0, 10800.0, 9840.0, 7920.0, 7440.0, 7260.0, 5820.0, 5820.0, 4860.0, 4860.0, 4560.0, 4440.0, 4080.0, 4020.0, 3360.0, 3300.0, 3240.0, 3120.0, 3060.0, 3060.0, 3000.0, 2940.0, 2880.0, 2880.0, 2760.0, 2760.0, 2700.0, 2640.0, 2520.0, 2460.0, 2460.0, 2400.0, 2160.0, 2100.0, 2100.0, 2040.0, 1980.0, 1920.0, 1920.0, 1680.0, 1680.0, 1680.0, 1620.0, 1560.0, 1500.0, 1320.0, 1200.0, 1080.0, 1020.0, 1020.0, 900.0, 900.0, 

In [4]:
texts = df['text']
texts = texts.fillna("")
# texts=[text.encode('ascii') for text in texts]
#print(texts)
comments = df['comment']
#print(comments)
timeInfo = time1_list
#print(timeInfo)
postInfo = df['poster']
#print(postInfo)

labels = df['lable']
#print(labels)
b = np.zeros([len(timeInfo), len(max(timeInfo, key=lambda x: len(x)))])
for i, j in enumerate(timeInfo):
    b[i][0:len(j)] = j
timeInfo = b
time_size = len(np.unique(timeInfo))
MAX_SENTS = len(timeInfo[0])  ####number of sentences

postInfo = np.array(postInfo)
#print(postInfo)
post_size = len(np.unique(postInfo))
#tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
#tokenizer.fit_on_texts(texts)
#print(MAX_SENTS)
data = np.zeros((len(texts), MAX_SENTS, MAX_SENT_LENGTH+1), dtype='int32')
#print(tokenizer.word_index['club'])

texts = texts.apply(lambda x: list(jieba.cut(x)))
word_count = Counter()
for text_seg in texts:
    word_count.update(text_seg)
word_list = sorted(word_count.items(), key=lambda x: x[1], reverse=True)
vocab = {}
for i, (word, freq) in enumerate(word_list):
    vocab[word] = i

#comments = comments.apply(lambda x: list(jieba.cut(x)))
#comments = comments.apply(lambda x: [vocab[word] for word in x if word in vocab])
m = 0
n = 0
for m, sentences in enumerate(comments):
    sentences = sentences.split(',')
    for n, sent in enumerate(sentences):
        if n < MAX_SENTS:
            k = 0
            sent = list(jieba.cut(sent))
            for word in sent:
                if word in vocab:
                    data[m, n, k] = vocab[word]
                    k = k + 1
                if k >= MAX_SENT_LENGTH:
                    break

word_index = vocab
print('Total %s unique tokens.' % len(word_index))
single_label = np.asarray(labels)
labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

###standardize
timeInfo = preprocessing.StandardScaler().fit_transform(timeInfo)
postInfo = preprocessing.StandardScaler().fit_transform(postInfo.reshape(-1, 1))
HAN_pre = []
HAN_reca = []
HAN_f1 = []
HAN_AUC = []
HAN_TIME = []
embeddings_index = Word2Vec.load_word2vec_format("word2vec_model.bin", binary=True, )  #

# print('Total %s word vectors.' % len(embeddings_index))
embedding_matrix = np.random.random((len(word_index) + 1, POST_DIM))
outword_dic = dict()
for word, i in word_index.items():
    if word in embeddings_index.vocab:
        embedding_vector = embeddings_index[word]
        embedding_matrix[i] = embedding_vector
    else:
        new_vector = np.random.rand(POST_DIM, )
        outword_dic.setdefault(word, new_vector)
        embedding_matrix[i] = outword_dic[word]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.802 seconds.
Prefix dict has been built successfully.


Total 7189 unique tokens.
Shape of data tensor: (825, 490, 31)
Shape of label tensor: (825, 2)


C:\Users\Administrator\Desktop\keyan dateset\word2vecReader.py:166: RuntimeWarning: overflow encountered in square
  self.syn0norm = (self.syn0 / sqrt((self.syn0 ** 2).sum(-1))[..., newaxis]).astype(REAL)
C:\Users\Administrator\Desktop\keyan dateset\word2vecReader.py:166: RuntimeWarning: invalid value encountered in square
  self.syn0norm = (self.syn0 / sqrt((self.syn0 ** 2).sum(-1))[..., newaxis]).astype(REAL)
E:\anaconda\lib\site-packages\numpy\core\_methods.py:48: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
C:\Users\Administrator\Desktop\keyan dateset\word2vecReader.py:166: RuntimeWarning: invalid value encountered in divide
  self.syn0norm = (self.syn0 / sqrt((self.syn0 ** 2).sum(-1))[..., newaxis]).astype(REAL)


In [ ]:
for j in range(10):
    start_time = process_time()
    indices = np.arange(data.shape[0])
    np.random.shuffle(indices)
    data1 = data[indices]
    print(data1.shape)
    labels1 = labels[indices]
    single_label1=single_label[indices]
    timeInfo1=timeInfo[indices]
    print(timeInfo1.shape)
    timeInfo1=timeInfo1.reshape((825,MAX_SENTS,1))
    print(timeInfo1.shape)
    data1=np.dstack((data1, timeInfo1))
    postInfo1=postInfo[indices]
    print(postInfo1.shape)
    nb_validation_samples = int(VALIDATION_SPLIT * data1.shape[0])
    zeros=np.zeros(825)
    zeros=zeros.reshape((825,1,1))

    x_train = data1[:-nb_validation_samples]
    y_train = labels1[:-nb_validation_samples]
    zeros_train=zeros[:-nb_validation_samples]
    time_train=timeInfo1[:-nb_validation_samples]
    post_train=postInfo1[:-nb_validation_samples]
    median_value1 = np.median(post_train)
    post_train = np.where(np.isnan(post_train), 0, post_train)
    x_val = data1[-nb_validation_samples:]
    y_val = labels1[-nb_validation_samples:]
    zeros_test=zeros[-nb_validation_samples:]
    time_test=timeInfo1[-nb_validation_samples:]
    post_test=postInfo1[-nb_validation_samples:]
    median_value2 = np.median(post_test)
    post_test = np.where(np.isnan(post_test), 0, post_test)
    y_single=single_label1[-nb_validation_samples:]
    print("y_single: ",y_single)

    # break
    print('Number of positive and negative posts in training and test set')
    print (y_train.sum(axis=0))
    print (y_val.sum(axis=0))

    # building Hierachical Attention network

    embedding_layer = Embedding(len(word_index) + 1,
                                POST_DIM,
                                weights=[embedding_matrix],
                                input_length=MAX_SENT_LENGTH,
                                trainable=True,
                                mask_zero=True)

    all_input = Input(shape=(MAX_SENT_LENGTH+2,))
    sentence_input=crop(1, 0, MAX_SENT_LENGTH)(all_input)##slice
    time_input=crop(1, MAX_SENT_LENGTH, MAX_SENT_LENGTH+2)(all_input)##slice
    embedded_sequences = embedding_layer(sentence_input)
    l_lstm = Bidirectional(GRU(200, return_sequences=True))(embedded_sequences)
    l_att = AttLayer(200)(l_lstm)####(?,200)
    #time_embedding=Dense(TIME_DIM,activation='sigmoid')(time_input)
    merged_output=Concatenate()([l_att,time_input])###text+time information
    sentEncoder = Model(all_input, merged_output)
    review_input = Input(shape=(MAX_SENTS, MAX_SENT_LENGTH+2))
    norm_review_input = BatchNormalization()(review_input)
    review_encoder =TimeDistributed(sentEncoder)(norm_review_input)  # <==

    l_lstm_sent = Bidirectional(GRU(200, return_sequences=True))(review_encoder)
    #l_lstm_sent = Bidirectional(GRU(200, return_sequences=True))(review_input)
    #pred_time=Dense(1,activation='relu')(l_lstm_sent)
    fully_sent=Dense(1,use_bias=False)(l_lstm_sent)
    norm_fullysent=BatchNormalization()(fully_sent)
    pred_time=Activation(activation='linear')(norm_fullysent)

    zero_input=Input(shape=(1,1))
    shift_predtime=Concatenate(axis=1)([zero_input,pred_time])
    shift_predtime=crop(1, 0, MAX_SENTS)(shift_predtime)
    l_att_sent = AttLayer(200)(l_lstm_sent)

    ###embed the #likes, shares
    # post_input=Input(shape=(4,))
    # post_input = Input(shape=(None,4))    # <==
    post_input = Input(shape=(1,))    # <==
    #post_embedding = Dense(INFO_DIM, activation='sigmoid')(post_input)
    fully_post=Dense(INFO_DIM,use_bias=False)(post_input)
    norm_fullypost=BatchNormalization()(fully_post)
    post_embedding=Activation(activation='relu')(fully_post)
    # print("l_att_sent: ")
    # print(l_att_sent)
    print("post_embedding: ")
    print(post_embedding)
    x = concatenate([l_att_sent,post_embedding])###merge the document level vectro with the additional embedded features such as #likes
    fully_review=Dense(2,use_bias=False)(x)
    norm_fullyreview=BatchNormalization()(fully_review)
    preds=Activation(activation='softmax')(norm_fullyreview)

    # rmsprop = optimizers.adam(lr=0.001)
    rmsprop = optimizers.Adam(learning_rate=0.001,decay=0.99)   # <==

    # print("review_input: ", review_input)
    # print("post_input: ", post_input)
    # print("zero_input: ", zero_input)

    model = Model(inputs=[review_input,post_input,zero_input], outputs=[preds,shift_predtime])
    print(model.summary())
    model.compile(loss=['binary_crossentropy','mse'],loss_weights=[1,0.00002],
                  optimizer=rmsprop)
    #filepath = "weights/weights-improvement-{epoch:02d}-{loss:.2f}.hdf5"
    #checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
    #callbacks_list = [EarlyStopping(monitor='loss', patience=1,mode='min'),checkpoint]

    print("model fitting - Hierachical attention network for cyberbullying detection")

    model.fit([x_train,post_train,zeros_train], [y_train,time_train], batch_size=64,
              epochs=25,verbose=1)
    yp = model.predict([x_val,post_test,zeros_test], verbose=1)
    ypreds=yp[0]
    ypreds = np.argmax(ypreds, axis=1)
    #print y_single
    #print ypred
    f1=precision_recall_fscore_support(y_single.astype(int), ypreds)   # <==
    auc=roc_auc_score(y_single.astype('int'), ypreds)  #<==
    f1 = precision_recall_fscore_support(y_single.astype(int), ypreds)  # <==
    auc = roc_auc_score(y_single.astype('int'), ypreds)  # <== category
    end_time = process_time()
    cpu_time = end_time - start_time
    print(cpu_time)
    print(f1)
    print(auc)
    HAN_TIME.append(cpu_time)
    HAN_AUC.append(auc)
    HAN_f1.append(f1[2][1])
    HAN_reca.append(f1[1][1])
    HAN_pre.append(f1[0][1])

    #for t-sne visualization
    # if j==0:
    #     a=model.layers
    #     get_representations_test = K.function([model.layers[0].input,model.layers[1].input,model.layers[12].input], [model.layers[6].output])
    #     representations_test = get_representations_test([x_val,post_test,zeros_test])[0]
    #     representation_dict = {
    #         'representations': representations_test,
    #         'labels': y_single
    #     }
    #
    #     with open('HANCD_Tem_results.pickle', 'wb') as handle:
    #         pickle.dump(representation_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    # K.clear_session()

print(HAN_AUC)
print(HAN_f1)
print(HAN_pre)
print(HAN_reca)
print(HAN_TIME)
print ("TIME",np.mean(HAN_TIME), np.std(HAN_TIME))
print ("AUC",np.mean(HAN_AUC), np.std(HAN_AUC))
print ("f1", np.mean(HAN_f1), np.std(HAN_f1))
print ("precision",np.mean(HAN_pre), np.std(HAN_pre))
print ("recall", np.mean(HAN_reca), np.std(HAN_reca))


# In[ ]:

(825, 490, 31)
(825, 490)
(825, 490, 1)
(825, 1)
y_single:  [1. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 1. 0. 0.
 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0.
 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1.
 1. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0.
 0. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 0.]
Number of positive and negative posts in training and test set
[391. 269.]
[107.  58.]


E:\anaconda\lib\site-packages\keras\initializers\initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


post_embedding: 
KerasTensor(type_spec=TensorSpec(shape=(None, 30), dtype=tf.float32, name=None), name='activation_1/Relu:0', description="created by layer 'activation_1'")
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 490, 32)]    0           []                               
                                                                                                  
 batch_normalization (BatchNorm  (None, 490, 32)     128         ['input_2[0][0]']                
 alization)                                                                                       
                                                                                                  
 time_distributed (TimeDistribu  (None, 490, 402)    1354770     ['batch_normalization[0][0]']    
 ted)             